<a href="https://colab.research.google.com/github/Ayrsz/SignalAndSistemyProject/blob/main/Augmentation_Audios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install sox libsox-fmt-all


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libao-common libao4 libid3tag0 libmad0 libopencore-amrnb0 libopencore-amrwb0
  libsox-fmt-alsa libsox-fmt-ao libsox-fmt-base libsox-fmt-mp3 libsox-fmt-oss
  libsox-fmt-pulse libsox3 libwavpack1
Suggested packages:
  libaudio2 libsndio6.1
The following NEW packages will be installed:
  libao-common libao4 libid3tag0 libmad0 libopencore-amrnb0 libopencore-amrwb0
  libsox-fmt-all libsox-fmt-alsa libsox-fmt-ao libsox-fmt-base libsox-fmt-mp3
  libsox-fmt-oss libsox-fmt-pulse libsox3 libwavpack1 sox
0 upgraded, 16 newly installed, 0 to remove and 29 not upgraded.
Need to get 800 kB of archives.
After this operation, 2,533 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libao-common all 1.2.2+20180113-1.1ubuntu3 [6,568 B]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libao4 amd64 1.2.

In [ ]:
#Model CNN
import torch
import torchaudio

#Scientific computation
import numpy as np
import jax

#Plot and view
from matplotlib import pyplot as plt
from IPython import display

#Data manipulation
import pandas as pd
import scipy as sci
import os


In [ ]:
print(torch.__version__)

2.6.0+cu124


In [ ]:
from zipfile import ZipFile
from google.colab import drive
drive.mount("/content/drive", True)

Mounted at /content/drive


In [ ]:
#Unzip in a specific folder
with ZipFile("/content/drive/MyDrive/Datasets/DatasetAudios/GTZAN.zip", 'r') as zip_archive:
  zip_archive.extractall("/content/dataset/")

### Writing on dir :O

In [ ]:
#Return all directories genres
def dir_genres(path_all_genres : str):
  return [path_all_genres + '/' + pasta for pasta in os.listdir(path_all_genres)]

#Return all files.wav separated per genres
def dir_files_wav(path_all_genres : str):
  dir_pastas = dir_genres(path_all_genres)
  return [[dir + '/' + arquivo for arquivo in os.listdir(dir)] for dir in dir_pastas]

#Writing new audios
def write_new_files_wav(path_genre : str, audios, sample_rate):
  wavs = dir_files_wav(path_genre)
  if len(wavs) == 0:
    numero = 0
    genero = "teste"

  else:
    tamanho = len(wavs)
    genero = "teste"
    numero = int(tamanho - 1)

  numero_str = ("{:05d}".format(numero))
  nome_arquivo = genero + "." + numero_str + ".wav"

  for audio in audios:
    nome_arquivo = genero + "." + numero_str + ".wav"

    torch.save(audio, nome_arquivo, 2400)


    numero = numero + 1

In [ ]:
  def get_effects():
      return [["rate", "16000"], ["reverb", "50"]]

### Augmentations


In [ ]:
class augmentation_dataset:
  def __init__(self, path_all_genres : str):
    self.path_genres = dir_genres(path_all_genres)
    self.path_files = dir_files_wav(path_all_genres)

  @staticmethod
  def get_effects():
        return [["adelay=1500|0|500"]]

  def apply_effects(self, audio, sample_rate):  # 🔥 Agora é um método da classe
        efeitos = self.get_effects()  # Aqui pode chamar self.get_effects() se quiser
        new_audios, new_sample_rates = torchaudio.io.AudioEffector(efeitos)
        return new_audios, new_sample_rates

  def augmentation_audios_in_a_dir(self, index : int):
    for audio_path in self.path_files[index]:


        musica, sample_rate = torchaudio.load(audio_path)
        print(musica.shape, sample_rate)
        mod = self.apply_effects(musica, sample_rate)
        break

  def augmentation(self):

    for i in range(len(self.path_genres)):
      novos_audios = self.augmentation_audios_in_a_dir(i)



dataset = augmentation_dataset("/content/dataset/Data/genres_original")
#dataset.get_effects()
dataset.augmentation()

torch.Size([1, 661794]) 22050


OSError: libsox.so: cannot open shared object file: No such file or directory

In [ ]:
from IPython.display import Audio

from torchaudio.utils import download_asset

#Retorna string com diretorio .wav
SAMPLE_WAV = download_asset("tutorial-assets/steam-train-whistle-daniel_simon.wav")

SAMPLE_RIR = download_asset("tutorial-assets/Lab41-SRI-VOiCES-rm1-impulse-mc01-stu-clo-8000hz.wav")
SAMPLE_SPEECH = download_asset("tutorial-assets/Lab41-SRI-VOiCES-src-sp0307-ch127535-sg0042-8000hz.wav")
SAMPLE_NOISE = download_asset("tutorial-assets/Lab41-SRI-VOiCES-rm1-babb-mc01-stu-clo-8000hz.wav")
# Load the data
waveform1, sample_rate = torchaudio.load(SAMPLE_WAV, channels_first=False)

# Define effects
effect = ",".join(
    [
        "lowpass=frequency=300:poles=1",  # apply single-pole lowpass filter
        "atempo=0.8",  # reduce the speed
        "aecho=in_gain=0.8:out_gain=0.9:delays=200:decays=0.3|delays=400:decays=0.3"
        # Applying echo gives some dramatic feeling
    ],
)


# Apply effects
def apply_effect(waveform, sample_rate, effect):
    effector = torchaudio.io.AudioEffector(effect=effect)
    return effector.apply(waveform, sample_rate)


waveform2 = apply_effect(waveform1, sample_rate, effect)

print(waveform1.shape, sample_rate)
print(waveform2.shape, sample_rate)

In [ ]:
# Load the data
waveform1, sample_rate = torchaudio.load(SAMPLE_WAV, channels_first=False)

# Define effects
effect = ",".join(
    [
        "lowpass=frequency=300:poles=1",  # apply single-pole lowpass filter
        "atempo=0.8",  # reduce the speed
        "aecho=in_gain=0.8:out_gain=0.9:delays=200:decays=0.3|delays=400:decays=0.3"
        # Applying echo gives some dramatic feeling
    ],
)


# Apply effects
def apply_effect(waveform, sample_rate, effect):
    effector = torchaudio.io.AudioEffector(effect=effect)
    return effector.apply(waveform, sample_rate)


waveform2 = apply_effect(waveform1, sample_rate, effect)

print(waveform1.shape, sample_rate)
print(waveform2.shape, sample_rate)

In [ ]:
def plot_waveform(waveform, sample_rate, title="Waveform", xlim=None):
    waveform = waveform.numpy()

    num_channels, num_frames = waveform.shape
    time_axis = torch.arange(0, num_frames) / sample_rate

    figure, axes = plt.subplots(num_channels, 1)
    if num_channels == 1:
        axes = [axes]
    for c in range(num_channels):
        axes[c].plot(time_axis, waveform[c], linewidth=1)
        axes[c].grid(True)
        if num_channels > 1:
            axes[c].set_ylabel(f"Channel {c+1}")
        if xlim:
            axes[c].set_xlim(xlim)
    figure.suptitle(title)

In [ ]:
def plot_specgram(waveform, sample_rate, title="Spectrogram", xlim=None):
    waveform = waveform.numpy()

    num_channels, _ = waveform.shape

    figure, axes = plt.subplots(num_channels, 1)
    if num_channels == 1:
        axes = [axes]
    for c in range(num_channels):
        axes[c].specgram(waveform[c], Fs=sample_rate)
        if num_channels > 1:
            axes[c].set_ylabel(f"Channel {c+1}")
        if xlim:
            axes[c].set_xlim(xlim)
    figure.suptitle(title)

In [ ]:
plot_waveform(waveform1.T, sample_rate, title="Original", xlim=(-0.1, 3.2))
plot_specgram(waveform1.T, sample_rate, title="Original", xlim=(0, 3.04))
Audio(waveform1.T, rate=sample_rate)

In [ ]:
plot_waveform(waveform2.T, sample_rate, title="Effects Applied", xlim=(-0.1, 3.2))
plot_specgram(waveform2.T, sample_rate, title="Effects Applied", xlim=(0, 3.04))
Audio(waveform2.T, rate=sample_rate)